# TM10007 Assignment template

In [2]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/Fjoru/TM10007_PROJECT

In [ ]:
## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [3]:
# Data loading functions. Uncomment the one you want to use
from brats.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
print(data.index)

The number of samples: 167
The number of columns: 725
Index(['TCGA-02-0006', 'TCGA-02-0009', 'TCGA-02-0011', 'TCGA-02-0027',
       'TCGA-02-0033', 'TCGA-02-0034', 'TCGA-02-0037', 'TCGA-02-0046',
       'TCGA-02-0047', 'TCGA-02-0054',
       ...
       'TCGA-HT-7686', 'TCGA-HT-7690', 'TCGA-HT-7694', 'TCGA-HT-7879',
       'TCGA-HT-7884', 'TCGA-HT-8018', 'TCGA-HT-8111', 'TCGA-HT-8114',
       'TCGA-HT-8563', 'TCGA-HT-A61A'],
      dtype='object', name='ID', length=167)
